# Introduction

**IMPORTANT: Notebook does not work at the moment. We have only started implementing the Transformer model.**

This notebook has been created as part of the Music Generation project by Dmytro Lopushanskyy and Yana Muliarska.

The following sources have been used:

[How to utilize the Lakh MIDI Dataset, with examples](https://nbviewer.org/github/craffel/midi-dataset/blob/master/Tutorial.ipynb) 

[Generating Music Using Deep Learning](https://towardsdatascience.com/generating-music-using-deep-learning-cb5843a9d55e)  

[Using Deep Learning to generate pop music!](https://github.com/bearpelican/musicautobot)

[MusicGenDL](https://github.com/thamsuppp/MusicGenDL)

[The Lakh MIDI Dataset v0.1](https://colinraffel.com/projects/lmd/)

In [ ]:
!wget "http://hog.ee.columbia.edu/craffel/lmd/lmd_matched_h5.tar.gz" -P "/content/drive/MyDrive/music_generation/"

--2022-04-27 10:40:39--  http://hog.ee.columbia.edu/craffel/lmd/lmd_matched_h5.tar.gz
Resolving hog.ee.columbia.edu (hog.ee.columbia.edu)... 128.59.66.5
Connecting to hog.ee.columbia.edu (hog.ee.columbia.edu)|128.59.66.5|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6635217072 (6.2G) [application/x-gzip]
Saving to: ‘/content/drive/MyDrive/music_generation/lmd_matched_h5.tar.gz’

lmd_matched_h5.tar. 100%[===================>]   6.18G  11.2MB/s    in 9m 32s  

2022-04-27 10:50:11 (11.1 MB/s) - ‘/content/drive/MyDrive/music_generation/lmd_matched_h5.tar.gz’ saved [6635217072/6635217072]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!tar -xf "/content/drive/MyDrive/music_generation/lmd_matched_h5.tar.gz" -C "/content/"

In [ ]:
!tar -xf "/content/drive/MyDrive/music_generation/lmd_matched.tar.gz" -C "/content/"

In [ ]:
!tar -xf "/content/drive/MyDrive/music_generation/lpd_5_cleansed.tar.gz" -C "/content/"

In [ ]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install music21
!pip install pypianoroll
!pip install musicautobot
!pip install pebble

W: GPG error: https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease: The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
E: The repository 'https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease' is no longer signed.
     |████████████████████████████████| 5.6 MB 5.0 MB/s 
     |████████████████████████████████| 51 kB 5.9 MB/s 


**Imports**

In [ ]:
import os
import shutil
import glob
import numpy as np 
import pandas as pd
import pretty_midi
import pypianoroll
import tables
from music21 import *
import music21
import librosa
import librosa.display
import matplotlib.pyplot as plt
from keras.utils import np_utils
import json
import IPython.display
from datetime import datetime
from google.colab import files

from musicautobot.numpy_encode import *
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.multitask_transformer import *
from musicautobot.utils import midifile

import random
import itertools

root_dir = '/content'
drive_dir = '/content/drive/MyDrive/music_generation/'
data_dir = root_dir + '/lpd_5/lpd_5_cleansed'
music_dataset_lpd_dir = root_dir + '/lmd_matched'

**Getting MIDI and Song Metadata**

In [ ]:
# Utility functions for retrieving paths
def msd_id_to_dirs(msd_id):
    """Given an MSD ID, generate the path prefix.
    E.g. TRABCD12345678 -> A/B/C/TRABCD12345678"""
    return os.path.join(msd_id[2], msd_id[3], msd_id[4], msd_id)


def msd_id_to_h5(msd_id):
    """Given an MSD ID, return the path to the corresponding h5"""
    return os.path.join(root_dir, 'lmd_matched_h5',
                        msd_id_to_dirs(msd_id) + '.h5')

# Load the midi npz file from the LMD cleansed folder
def get_midi_npz_path(msd_id, midi_md5):
    return os.path.join(data_dir, msd_id_to_dirs(msd_id), midi_md5 + '.npz')
    
# Load the midi file from the Music Dataset folder
def get_midi_path(msd_id, midi_md5):
    return os.path.join(music_dataset_lpd_dir,
                        msd_id_to_dirs(msd_id), midi_md5 + '.mid')
    
# Open the cleansed ids - cleansed file ids : msd ids
cleansed_ids = pd.read_csv(os.path.join(drive_dir, 'cleansed_ids.txt'), delimiter = '    ', header = None)
lpd_to_msd_ids = {a:b for a, b in zip(cleansed_ids[0], cleansed_ids[1])}
msd_to_lpd_ids = {a:b for a, b in zip(cleansed_ids[1], cleansed_ids[0])}

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


**Reading Genre Annotations**

In [ ]:
genre_file_dir = os.path.join(drive_dir, 'msd_tagtraum_cd1.cls')
ids = []
genres = []
with open(genre_file_dir) as f:
    line = f.readline()
    while line:
        if line[0] != '#':
          split = line.strip().split("\t")
          if len(split) == 2:
            ids.append(split[0])
            genres.append(split[1])
          elif len(split) == 3:
            ids.append(split[0])
            ids.append(split[0])
            genres.append(split[1])
            genres.append(split[2])
        line = f.readline()
genre_df = pd.DataFrame(data={"TrackID": ids, "Genre": genres})
genre_dict = genre_df.groupby('TrackID')['Genre'].apply(lambda x: x.tolist()).to_dict()

In [ ]:
genre_dict_filtered = dict((key, value) for key, value in genre_dict.items() if key in msd_to_lpd_ids)

**Retrieving MSD IDs**

In [ ]:
# Get all song MSD IDs in pop rock genre
pop_song_msd_ids = [k for k, v in genre_dict_filtered.items() if 'Pop_Rock' in v]

# Randomly choose 1000 songs out of these
train_ids = random.choices(pop_song_msd_ids, k = 1000)

**Transformer Code**

In [ ]:
import pebble
from musicautobot.numpy_encode import *
from musicautobot.utils.file_processing import process_all, process_file
from musicautobot.config import *
from musicautobot.music_transformer import *


# Location of your midi files
midi_path = Path(root_dir + '/lmd_matched/A/A/A')
data_path = Path(root_dir)
data_save_name = 'musicitem_data_save.pkl'


midi_files = get_files(midi_path, '.mid', recurse=True)
data_mj = MusicDataBunch.from_files(midi_files, data_path, processors=[Midi2ItemProcessor()], bs=4, bptt=128, encode_position=False)

learn_mj = music_model_learner(data_mj, arch=TransformerXL, config=default_config())

import warnings
warnings.simplefilter("ignore", UserWarning)

learn_mj.fit_one_cycle(5)


/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(a, dtype=dtype, **kwargs)


epoch,train_loss,valid_loss,accuracy,time
0,2.300060,2.679075,0.381539,00:58
1,2.118613,3.089572,0.339334,00:58
2,2.078274,3.040560,0.349270,00:59
3,1.921540,2.883530,0.382388,00:59
4,1.909261,2.693087,0.364555,00:59


AttributeError: ignored

In [ ]:
midi_file = Path(root_dir + '/lmd_matched/A/A/A/TRAAAZF12903CCCF6B/05f21994c71a5f881e64f45c8d706165.mid')
item = MusicItem.from_file(midi_file, data_mj.vocab);
pred = learn_mj.predict(item, n_words=500)

pred[0].to_stream().show('text')

pred[0].play()

{0.0} <music21.meter.TimeSignature 4/4>
{0.0} <music21.tempo.MetronomeMark animato Quarter=120>
{0.0} <music21.key.KeySignature of no sharps or flats>
{0.0} <music21.stream.Part 0x7ff42a8532d0>
    {0.0} <music21.instrument.Piano Piano>
    {0.5} <music21.chord.Chord E-2 G3 F#4>
    {1.0} <music21.chord.Chord F2 C4 D4 E4>
    {1.5} <music21.chord.Chord C4 E4 F4 G4 A4 B-4 C5>
    {1.75} <music21.chord.Chord G3 D4 F4 G4 B-4 C5>
    {2.0} <music21.chord.Chord B-3 C4 D4 E4 G4 A4 C5 D5>
    {2.25} <music21.chord.Chord C4 F4 F#4 G4>
    {2.5} <music21.chord.Chord B-3 C4 D4 E4 F#4 B-4>
    {2.75} <music21.chord.Chord G3 D4 E4>
    {3.0} <music21.note.Note F#>
    {3.25} <music21.chord.Chord B-3 F#4>
    {3.5} <music21.chord.Chord C4 D4 E4>
    {3.75} <music21.chord.Chord C4 G4>
    {4.0} <music21.chord.Chord B-3 E4 F4 G4 A4 B-4>
    {4.25} <music21.chord.Chord F2 C4 D4 E4 F#4 B-4 C5>
    {4.5} <music21.chord.Chord G3 C4 E4 F#4 G4 A4 B-4 C5>
    {4.75} <music21.chord.Chord C4 E4 A4 B-4 C5 D5>


NameError: ignored

In [ ]:
from IPython import display

midi_pred = pred[0].play()
pm = pretty_midi.PrettyMIDI(midi_pred)

_SAMPLING_RATE = 16000

def display_audio(pm: pretty_midi.PrettyMIDI, seconds=1):
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display.Audio(waveform_short, rate=_SAMPLING_RATE)

In [ ]:
instrument = pretty_midi.Instrument(
    program=pretty_midi.instrument_name_to_program(
        'Acoustic Grand Piano'))

pm.instruments.append(instrument)

In [ ]:
print('Number of instruments:', len(pm.instruments))
instrument = pm.instruments[0]
instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
print('Instrument name:', instrument_name)

Number of instruments: 1
Instrument name: Acoustic Grand Piano


In [ ]:
def midi_to_notes(midi_file: str) -> pd.DataFrame:
  pm = pretty_midi.PrettyMIDI(midi_file)
  instrument = pretty_midi.Instrument(
    program=pretty_midi.instrument_name_to_program(
        'Acoustic Grand Piano'))
  pm.instruments.append(instrument)
  instrument = pm.instruments[0]
  notes = collections.defaultdict(list)

  # Sort the notes by start time
  sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
  prev_start = sorted_notes[0].start

  for note in sorted_notes:
    start = note.start
    end = note.end
    notes['pitch'].append(note.pitch)
    notes['start'].append(start)
    notes['end'].append(end)
    notes['step'].append(start - prev_start)
    notes['duration'].append(end - start)
    prev_start = start

  return pd.DataFrame({name: np.array(value) for name, value in notes.items()})

In [ ]:
raw_notes = midi_to_notes(midi_pred)
raw_notes.head()

IndexError: ignored